Analysing the Medicaid and Chip Eligibilty

Import pandas Libriries

In [194]:
import pandas as pd

In [195]:
#convert the Data Dictionary csv file into .md file

In [196]:
Data_Dictionary = pd.read_csv("../data/Data_Dictionary.csv")
Data_Dictionary = Data_Dictionary.drop(columns=['name','type','format'])
#Convert the data frame into a mark down string(Github style)
Data_Dictionary = Data_Dictionary.to_markdown(tablefmt="github", index=False)
# Export the Markdown string to a .md file
output_path = "../Data_Dictionary.md"
with open(output_path, 'w', encoding='utf-8') as f:
  f.write(Data_Dictionary)

In [197]:
# Read the csv data into medicaid dataframe
medicaid = pd.read_csv("../data/medicaid.csv")
medicaid.head()

,State Abbreviation,State Name,Reporting Period,State Expanded Medicaid,Preliminary or Updated,Final Report,New Applications Submitted to Medicaid and CHIP Agencies,New Applications Submitted to Medicaid and CHIP Agencies - footnotes,Applications for Financial Assistance Submitted to the State Based Marketplace,Applications for Financial Assistance Submitted to the State Based Marketplace - footnotes,...,Total Medicaid and CHIP Determinations Processed between 31 days and 45 days,Total Medicaid and CHIP Determinations Processed between 31 days and 45 days - footnotes,Total Medicaid and CHIP Determinations Processed in More than 45 Days,Total Medicaid and CHIP Determinations Processed in More than 45 Days - footnotes,Total Call Center Volume (Number of Calls),Total Call Center Volume (Number of Calls) - footnotes,Average Call Center Wait Time (Minutes),Average Call Center Wait Time (Minutes) - footnotes,Average Call Center Abandonment Rate,Average Call Center Abandonment Rate - footnotes
0,AK,Alaska,201309,N,U,Y,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AK,Alaska,201706,Y,P,N,3017.0,Includes Renewals and/or Redeterminations,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AK,Alaska,201706,Y,U,Y,3069.0,Includes Renewals and/or Redeterminations,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AK,Alaska,201707,Y,P,N,3187.0,Includes Renewals and/or Redeterminations,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AK,Alaska,201707,Y,U,Y,3265.0,Includes Renewals and/or Redeterminations,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Clean the Data

In [198]:
#Clean column names
medicaid.columns = (
    medicaid.columns
    .str.lower()  #lower the alll column names
    .str.strip()  #Remove extra spaces
    .str.replace(' ','_') #replace space with underscore
    .str.replace(r"_+", '_') #replace multiple underscores, if have any
    .str.replace(r"_-_footnotes", '_footnotes')
)

In [199]:
#Change the data type of reporting_period and extract month
medicaid['reporting_period'] = pd.to_datetime(medicaid['reporting_period'].astype(str),
    format='%Y%m'
    )
medicaid['month_name']= medicaid['reporting_period'].dt.month_name()
medicaid['year'] = medicaid['reporting_period'].dt.year

In [200]:
#reorder columns
medicaid.insert(3, 'year', medicaid.pop('year'))  # pop removes the coumn from the location and insert allows to insert column in specific location 
medicaid.insert(4, 'month_name', medicaid.pop('month_name'))

In [202]:
medicaid[medicaid.duplicated(subset=['state_abbreviation', 'reporting_period'])]

,state_abbreviation,state_name,reporting_period,year,month_name,state_expanded_medicaid,preliminary_or_updated,final_report,new_applications_submitted_to_medicaid_and_chip_agencies,new_applications_submitted_to_medicaid_and_chip_agencies_footnotes,...,total_medicaid_and_chip_determinations_processed_between_31_days_and_45_days,total_medicaid_and_chip_determinations_processed_between_31_days_and_45_days_footnotes,total_medicaid_and_chip_determinations_processed_in_more_than_45_days,total_medicaid_and_chip_determinations_processed_in_more_than_45_days_footnotes,total_call_center_volume_(number_of_calls),total_call_center_volume_(number_of_calls)_footnotes,average_call_center_wait_time_(minutes),average_call_center_wait_time_(minutes)_footnotes,average_call_center_abandonment_rate,average_call_center_abandonment_rate_footnotes
2,AK,Alaska,2017-06-01,2017,June,Y,U,Y,3069.0,Includes Renewals and/or Redeterminations,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AK,Alaska,2017-07-01,2017,July,Y,U,Y,3265.0,Includes Renewals and/or Redeterminations,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,AK,Alaska,2017-08-01,2017,August,Y,U,Y,3967.0,Includes Renewals and/or Redeterminations,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,AK,Alaska,2017-09-01,2017,September,Y,U,Y,3553.0,Includes Renewals and/or Redeterminations,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,AK,Alaska,2017-10-01,2017,October,Y,U,Y,3574.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10292,WY,Wyoming,2025-05-01,2025,May,N,U,Y,1520.0,Includes Administrative Data Transfers; Does N...,...,119.0,NaN,295.0,NaN,10634.0,NaN,1.0,Call centers offer callbacks,0.010,Callbacks are included
10294,WY,Wyoming,2025-06-01,2025,June,N,U,Y,1429.0,Includes Administrative Data Transfers; Does N...,...,71.0,NaN,159.0,NaN,9879.0,NaN,1.0,Call centers offer callbacks,0.011,Callbacks are included
10296,WY,Wyoming,2025-07-01,2025,July,N,U,Y,1539.0,Includes Administrative Data Transfers; Does N...,...,115.0,NaN,170.0,NaN,11585.0,NaN,1.0,Call centers offer callbacks,0.002,Callbacks are included
10298,WY,Wyoming,2025-08-01,2025,August,N,U,Y,1396.0,Includes Administrative Data Transfers; Does N...,...,88.0,NaN,306.0,NaN,11046.0,NaN,1.0,Call centers offer callbacks,0.003,Callbacks are included


In [203]:
#Remove duplicate rows for state and reporting period
medicaid_clean = (
    medicaid
    .sort_values("preliminary_or_updated") #Sort values for preliminary_or_updated
    .drop_duplicates(subset=['state_abbreviation', 'reporting_period'], keep='last') #Make sure to keep the U (Upadated records) if there are priliminary and updated
    
)
medicaid_clean

,state_abbreviation,state_name,reporting_period,year,month_name,state_expanded_medicaid,preliminary_or_updated,final_report,new_applications_submitted_to_medicaid_and_chip_agencies,new_applications_submitted_to_medicaid_and_chip_agencies_footnotes,...,total_medicaid_and_chip_determinations_processed_between_31_days_and_45_days,total_medicaid_and_chip_determinations_processed_between_31_days_and_45_days_footnotes,total_medicaid_and_chip_determinations_processed_in_more_than_45_days,total_medicaid_and_chip_determinations_processed_in_more_than_45_days_footnotes,total_call_center_volume_(number_of_calls),total_call_center_volume_(number_of_calls)_footnotes,average_call_center_wait_time_(minutes),average_call_center_wait_time_(minutes)_footnotes,average_call_center_abandonment_rate,average_call_center_abandonment_rate_footnotes
10301,WY,Wyoming,2025-10-01,2025,October,N,P,N,1473.0,Includes Administrative Data Transfers; Does N...,...,165.0,NaN,247.0,NaN,12623.0,NaN,1.0,Call centers offer callbacks,0.000,Callbacks are included
4241,MD,Maryland,2025-10-01,2025,October,Y,P,N,10499.0,NaN,...,21.0,NaN,0.0,NaN,65506.0,Does not include all calls received by call ce...,2.0,Call centers offer callbacks; Does not include...,0.050,Does not include all calls received by call ce...
4039,MA,Massachusetts,2025-10-01,2025,October,Y,P,N,14018.0,NaN,...,14.0,NaN,7.0,NaN,249083.0,Does not include all calls received after busi...,2.0,Call centers offer callbacks; Does not include...,0.027,Does not include all calls received after busi...
8281,SC,South Carolina,2025-10-01,2025,October,N,P,N,15978.0,NaN,...,425.0,NaN,258.0,NaN,130771.0,Does not include all calls received after busi...,1.0,Callbacks are included; Call centers offer cal...,0.023,Callbacks are included; Does not include all c...
4847,MN,Minnesota,2025-10-01,2025,October,Y,P,N,5939.0,NaN,...,0.0,NaN,0.0,NaN,23919.0,Does not include all calls received by call ce...,5.0,Does not include all calls received by call ce...,0.210,Does not include all calls received by call ce...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4566,MI,Michigan,2022-06-01,2022,June,Y,U,Y,22107.0,NaN,...,2206.0,NaN,378.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4564,MI,Michigan,2022-05-01,2022,May,Y,U,Y,20455.0,NaN,...,2060.0,NaN,463.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4562,MI,Michigan,2022-04-01,2022,April,Y,U,Y,20385.0,NaN,...,2284.0,NaN,723.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4558,MI,Michigan,2022-02-01,2022,February,Y,U,Y,20034.0,NaN,...,3723.0,NaN,735.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [188]:
medicaid_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5202 entries, 10301 to 5150
Data columns (total 46 columns):
 #   Column                                                                                    Non-Null Count  Dtype         
---  ------                                                                                    --------------  -----         
 0   state_abbreviation                                                                        5202 non-null   object        
 1   state_name                                                                                5202 non-null   object        
 2   reporting_period                                                                          5202 non-null   datetime64[ns]
 3   year                                                                                      5202 non-null   int32         
 4   month_name                                                                                5202 non-null   object        
 5   state_e

In [178]:
#Drop all the footnotes columns
medicaid_drop_footnotes = medicaid.drop(columns= medicaid.columns[medicaid.columns.str.contains("_footnotes")])

In [179]:
#export csv cleaned csv files
medicaid_drop_footnotes.to_csv("../data/medicaid_drop_footnotes.csv", index=False)
medicaid.to_csv("../data/medicaid_chip.csv", index=False)